# **Mulberry Leaf: Data Preparation**
### ***Dataset: Mulberry Leaf Dataset https://www.kaggle.com/datasets/nahiduzzaman13/mulberry-leaf-dataset***
### *Downloading dataset, preprocessing, splitting into 5-folds, and augmentation for each train split.*

In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
import tensorflow as tf

import shutil
import requests
import zipfile

from os import path, listdir, makedirs, rename, unlink
from  matplotlib import pyplot as plt
from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split, StratifiedKFold

from tensorflow.keras.preprocessing import image


In [2]:
def gamma_correction(img, gamma=2.0):
    """
    Applies gamma correction to darken bright areas of an RGB image.
    A gamma value less than 1 darkens bright areas.

    Parameters:
    - img: Input RGB image (numpy array).
    - gamma: Gamma value (less than 1 to darken bright areas).

    Returns:
    - Gamma-corrected RGB image.
    """
    # Convert the image to float32 for precision and normalize it to [0, 1]
    img_normalized = np.float32(img) / 255.0
    
    # Apply gamma correction to each channel separately
    img_corrected = np.power(img_normalized, gamma)
    
    # Scale back to [0, 255] and convert to uint8
    img_corrected = np.uint8(np.clip(img_corrected * 255, 0, 255))
    
    return img_corrected

def singleScaleRetinex(img, sigma):
    """
    Apply Single-Scale Retinex.
    """
    return np.log10(img + 1) - np.log10(cv.GaussianBlur(img, (0, 0), sigma) + 1)

def multiScaleRetinex(img, sigma_list):
    """
    Apply Multi-Scale Retinex.
    """
    retinex = np.zeros_like(img, dtype=np.float64)
    for sigma in sigma_list:
        retinex += singleScaleRetinex(img, sigma)
    return retinex / len(sigma_list)

def colorRestoration(img, alpha):
    """
    Apply Color Restoration (simplified).
    """
    img_sum = np.sum(img, axis=2, keepdims=True) + 1e-6  # Avoid division by zero
    return np.log10(alpha * img + 1) - np.log10(img_sum)

def simplestColorBalance(img, low_clip, high_clip):
    """
    Perform Simplest Color Balance.
    """
    total = img.shape[0] * img.shape[1]
    for i in range(img.shape[2]):
        unique, counts = np.unique(img[:, :, i], return_counts=True)
        current = 0
        for u, c in zip(unique, counts):
            if float(current) / total < low_clip:
                low_val = u
            if float(current) / total < high_clip:
                high_val = u
            current += c
        img[:, :, i] = np.clip(img[:, :, i], low_val, high_val)
    return img

def MSRCR(img, sigma_list=[15, 80, 250], G=1.0, alpha=25.0, low_clip=0.00001, high_clip=0.99999):
    """
    Multi-Scale Retinex with Color Restoration (MSRCR).

    Parameters:
        - img: Input image (uint8).
        - sigma_list: List of sigma values for Retinex scales.
        - G: Gain factor.
        - alpha: Scaling factor for color restoration.
        - low_clip: Lower percentile for simplest color balance.
        - high_clip: Upper percentile for simplest color balance.

    Returns:
        - Processed image (uint8).
    """
    img = np.float64(img) + 1.0  # Convert to float for processing

    # Apply Multi-Scale Retinex
    img_retinex = multiScaleRetinex(img, sigma_list)

    # Apply Color Restoration
    img_color = colorRestoration(img, alpha)

    # Combine Retinex and Color Restoration
    img_msrcr = G * img_retinex * img_color

    # Normalize each channel to [0, 255]
    for i in range(img_msrcr.shape[2]):
        img_msrcr[:, :, i] = (img_msrcr[:, :, i] - np.min(img_msrcr[:, :, i])) / \
                             (np.max(img_msrcr[:, :, i]) - np.min(img_msrcr[:, :, i])) * 255

    # Clip to valid range and convert to uint8
    img_msrcr = np.uint8(np.clip(img_msrcr, 0, 255))

    # Apply Simplest Color Balance
    img_msrcr = simplestColorBalance(img_msrcr, low_clip, high_clip)

    return img_msrcr


In [3]:
plt.rcParams['font.family'] = 'serif'            # Use a specific font
plt.rcParams['font.size'] = 14                   # Set a general font size

def imshow(imgs, titles=None, color='gray'):
    '''Utility class for showing images.'''
    r, c = 4, 4
    # show all images
    plt.figure(figsize=(15, 12))
    for idx in range(1, min(r*c, len(imgs))+1):
        plt.subplot(r, c, idx)
        plt.axis('off')
        if titles != None and idx-1 < len(titles):
            plt.title(titles[idx-1])
        plt.imshow(imgs[idx-1], cmap=color)    
    plt.show()

def preprocess_and_save(df_info, to_dir=None, to_msr_dir=None):
    '''Resize & save imgs into given directory; using label information.
    '''
    img_count = len(df_info)
    print(f'Found {img_count} image(s) in dataframe.')
    
    with tqdm(total=img_count) as pbar:
        for _, row in df_info.iterrows():
            img_path, label = row['path'], row['label']
            pbar.set_postfix({"Processing": path.basename(img_path)})
            img = np.array(cv.cvtColor(cv.imread(img_path, cv.IMREAD_COLOR), cv.COLOR_BGR2RGB), dtype=np.uint8)
            
            # Resize image to INPUT_SIZE
            rsz_img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
            
            # Save each patch using tf.keras.utils.save_img
            if to_dir:
                makedirs(path.join(to_dir, label), exist_ok=True)
                tf.keras.utils.save_img(path.join(to_dir, label, path.basename(img_path)), rsz_img)
            if to_msr_dir:
                msr_img = MSRCR(rsz_img, sigma_list=[12, 48, 144], G=1.5, alpha=150)
                msr_img = gamma_correction(msr_img, gamma=2.0)
                makedirs(path.join(to_msr_dir, label), exist_ok=True)
                tf.keras.utils.save_img(path.join(to_msr_dir, label, path.basename(img_path)), msr_img)
                
            pbar.update(1)
            pass

        if to_dir:
            print(f'Saved {img_count} image(s) into: {to_dir}')
        if to_msr_dir:
            print(f'Saved {img_count} image(s) into: {to_msr_dir}')

def random_transformations(image):
    # Rotate the image randomly by 0, 90, 180, or 270 degrees
    rotated_image = tf.image.rot90(image, k=np.random.choice([0, 1, 2, 3]))
    # Adjust brightness randomly
    bright_image = tf.image.random_brightness(rotated_image, max_delta=0.1)
    # Flip randomly
    flipped_image = tf.image.random_flip_left_right(bright_image)
    return flipped_image

def augment_images(cls_names, _train_dir, _AUG_IMG_COUNT=[4000, 4000]):
    _total_count = 0
    for idx, cls_name in enumerate(cls_names):
        # Define paths
        _target_train_dir = path.join(ROOT_OUTPUT_DIR, 'aug')
        _augmented_train_dir = path.join(_train_dir, cls_name)

        # Making subdirectory for classing
        _target_train_subdir = path.join(_target_train_dir, cls_name)

        shutil.rmtree(_target_train_dir, ignore_errors=True)

        # Make a copy of the original images to the target folder
        shutil.copytree(_augmented_train_dir, _target_train_subdir)

        # Define ImageDataGenerator for augmentation
        _datagen = image.ImageDataGenerator(
            preprocessing_function=random_transformations
        )

        # Flow images from the target directory, apply augmentation, and save to augmented directory
        _batch_size = 8
        _aug_gen = _datagen.flow_from_directory(
            _target_train_dir,
            target_size=(IMG_SIZE, IMG_SIZE),
            batch_size=_batch_size,
            color_mode='rgb', 
            class_mode='categorical',
            save_to_dir=_augmented_train_dir,
            save_prefix='augmented_',
            save_format='png'
        )

        # creating N augmented images per class
        _initial_img_count = len(listdir(_augmented_train_dir))
        _img_count = 0
        with tqdm(total=_AUG_IMG_COUNT[idx]) as bar:
            while True:
                next(_aug_gen)
                _img_count = len(listdir(_augmented_train_dir)) - _initial_img_count
                if _img_count >= _AUG_IMG_COUNT[idx]:
                    bar.update(_AUG_IMG_COUNT[idx] - bar.n)
                    break
                bar.update(_img_count - bar.n)

        _total_count = _total_count + _img_count
        print(f'Saved {_img_count} image(s) into: {_augmented_train_dir}')
        shutil.rmtree(_target_train_dir)
    
    print(f'Augmentation complete. Generated total {_total_count} image(s).')


# 1. Fetching Dataset

In [4]:
ROOT_OUTPUT_DIR = '/kaggle/working/'
ROOT_INPUT_DIR = '/kaggle/input/mulberry-leaf-dataset/Mulberry Data/'
NEW_DATA_DIR = path.join(ROOT_OUTPUT_DIR, 'data')
MSR_DATA_DIR = path.join(ROOT_OUTPUT_DIR, 'data_msr')

IMG_SIZE = 224

INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

DISPLAY_LABELS = ['Disease Free', 'Leaf Rust', 'Leaf Spot']

In [5]:
## Remove old generated files

# Iterate over each item in the directory
for filename in listdir(ROOT_OUTPUT_DIR):
    file_path = path.join(ROOT_OUTPUT_DIR, filename)
    try:
        # Check if it is a file or directory and remove accordingly
        if path.isfile(file_path) or path.islink(file_path):
            unlink(file_path)  # Remove the file or link
        elif path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove the directory
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')
print("Directory emptied.")


Directory emptied.


# 2. Organizing Data

In [6]:
df_lst = []
for cls_idx, dir_name in enumerate(listdir(ROOT_INPUT_DIR)):
    img_paths = sorted([str(file_path) for file_path in Path(path.join(ROOT_INPUT_DIR, dir_name)).iterdir() if file_path.suffix in ['.jpg', '.png', '.JPG', '.PNG']])
    label = DISPLAY_LABELS[cls_idx]
    df_lst.append(pd.DataFrame({"path": img_paths, "label": label}))

df = pd.concat(df_lst, ignore_index=True)
df.head()

,path,label
0,/kaggle/input/mulberry-leaf-dataset/Mulberry D...,Disease Free
1,/kaggle/input/mulberry-leaf-dataset/Mulberry D...,Disease Free
2,/kaggle/input/mulberry-leaf-dataset/Mulberry D...,Disease Free
3,/kaggle/input/mulberry-leaf-dataset/Mulberry D...,Disease Free
4,/kaggle/input/mulberry-leaf-dataset/Mulberry D...,Disease Free


In [7]:
df['label'].value_counts()

label
Leaf Rust       489
Disease Free    440
Leaf Spot       162
Name: count, dtype: int64

In [8]:
## Save dataframe to storage

makedirs(NEW_DATA_DIR, exist_ok=True)
df.to_csv(path.join(NEW_DATA_DIR, 'labels.csv'), index=False)
print(f"Saved to directory: {path.join(NEW_DATA_DIR, 'labels.csv')}")


Saved to directory: /kaggle/working/data/labels.csv


# 3. Generating Folds

In [9]:
## Generate folds & save patches

# Initialize StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

# Create folds
for fold, (train_idx, test_idx) in enumerate(kf.split(df, df['label'])):
    
    print(f'Fold #{fold+1}:')
    _fold_str = f'fold{fold+1}'
    
    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]

    print('\nTrain distribution:')
    print(train_df['label'].value_counts())
    print('\nTest distribution:')
    print(test_df['label'].value_counts())
    print('')
    
    print('Saving train & test images...')
    preprocess_and_save(train_df, to_dir=path.join(NEW_DATA_DIR, _fold_str, 'train'), 
                        to_msr_dir=path.join(MSR_DATA_DIR, _fold_str, 'train'))
    preprocess_and_save(test_df, to_dir=path.join(NEW_DATA_DIR, _fold_str, 'test'), 
                        to_msr_dir=path.join(MSR_DATA_DIR, _fold_str, 'test'))
    print('')
    
    print('Augmentation of train images...')
    augment_images(DISPLAY_LABELS, _train_dir=path.join(NEW_DATA_DIR, _fold_str, 'train'), _AUG_IMG_COUNT=[2148, 2109, 2371])
    print('Augmentation of MSR train images...')
    augment_images(DISPLAY_LABELS, _train_dir=path.join(MSR_DATA_DIR, _fold_str, 'train'), _AUG_IMG_COUNT=[2148, 2109, 2371])
    print('')
    
    # zipping & cleaning
    print('Archiving...')
    shutil.make_archive(path.join(NEW_DATA_DIR, _fold_str), 'zip', path.join(NEW_DATA_DIR, _fold_str))
    shutil.rmtree(path.join(NEW_DATA_DIR, _fold_str))
    shutil.make_archive(path.join(MSR_DATA_DIR, _fold_str), 'zip', path.join(MSR_DATA_DIR, _fold_str))
    shutil.rmtree(path.join(MSR_DATA_DIR, _fold_str))
    pass


Fold #1:

Train distribution:
label
Leaf Rust       391
Disease Free    352
Leaf Spot       129
Name: count, dtype: int64

Test distribution:
label
Leaf Rust       98
Disease Free    88
Leaf Spot       33
Name: count, dtype: int64

Saving train & test images...
Found 872 image(s) in dataframe.


100%|██████████| 872/872 [14:21<00:00,  1.01it/s, Processing=s (9).JPG]


Saved 872 image(s) into: /kaggle/working/data/fold1/train
Saved 872 image(s) into: /kaggle/working/data_msr/fold1/train
Found 219 image(s) in dataframe.


100%|██████████| 219/219 [03:31<00:00,  1.04it/s, Processing=s (63).JPG]


Saved 219 image(s) into: /kaggle/working/data/fold1/test
Saved 219 image(s) into: /kaggle/working/data_msr/fold1/test

Augmentation of train images...
Found 352 images belonging to 1 classes.


...

Saved 2152 image(s) into: /kaggle/working/data_msr/fold5/train/Disease Free
Found 392 images belonging to 1 classes.


100%|██████████| 2109/2109 [00:46<00:00, 45.60it/s]


Saved 2112 image(s) into: /kaggle/working/data_msr/fold5/train/Leaf Rust
Found 129 images belonging to 1 classes.


100%|██████████| 2371/2371 [00:51<00:00, 45.68it/s]


Saved 2378 image(s) into: /kaggle/working/data_msr/fold5/train/Leaf Spot
Augmentation complete. Generated total 6642 image(s).

Archiving...
